<p align="center">
  <a>
    <img src="./figures/logo-hi-paris-retina.png" alt="Logo" width="280" height="180">
  </a>

  <h3 align="center">Data Science Bootcamp</h3>
</p>

Authors : Yann Berthelot, Florian Bettini, Laure-Amélie Colin

### Resources

For more details about Data Cleaning, please refer to the `1_fires_preparation.ipynb` notebook

Objective of this lab
======

Merge external data (temperature, rainfall, demographics) into a single table. Clean the data to obtain a final dataset, without errors, duplicates, irrelevant values... ready to be analyzed


Objectives of this Notebook
======

##### **Objectives:**
- Read multiple external datasets
    - temperatures and precipitations from the `./data/1_raw/cities/` folder
    - demographics from the file `./data/1_raw/cities/us-cities-demographics.csv`
- Merge them into a unique table
- Save the cleaned DataFrame in `./data/2_clean/external_data.csv`

In [103]:
# first, we need to import pandas
import pandas as pd

##### **Data structure**:
- Temperature and precipitation
    - The `./data/1_raw/cities/` folder contains temperature and precipitation values for 210 US cities.
    - it contains a file `./data/1_raw/cities/city_info.csv` that maps the city name with a code (example: "USW00094728" for "New York"). Columns are:
        * `Name`: name of the city
        * `ID`: ID of the city (this ID is the name of the corresponding file with temperature and precipitation values)
        * `Lat`: Latitude
        * `Lon`: Longitude
        * `Stn.Name`: name of the city for reporting (not used)
        * `Stn.stDate`: minimum date from which temperature and precipitation are available
        * `Stn.edDate`: maximum date for data availability
        * all other columns are not relevant

In [104]:
# city infos
city_infos = pd.read_csv("./data/1_raw/cities/city_info.csv")
print('City infos: print the first 5 rows of the dataframe')
display(city_infos.head())

City infos: print the first 5 rows of the dataframe


,Unnamed: 0,Name,ID,Lat,Lon,Stn.Name,Stn.stDate,Stn.edDate,Unnamed: 8
0,1,Lander,USW00024021,42.8153,-108.7261,LANDER WBO,1892-01-01,5/28/1946,False
1,2,Lander,USW00024021,42.8153,-108.7261,LANDER HUNT FIELD,5/29/1946,12/31/2021,False
2,3,Cheyenne,USW00024018,41.1519,-104.8061,CHEYENNE WBO,1871-01-01,8/31/1935,False
3,4,Cheyenne,USW00024018,41.1519,-104.8061,CHEYENNE MUNICIPAL ARPT,9/1/1935,12/31/2021,False
4,5,Wausau,USW00014897,44.9258,-89.6256,Wausau Record Herald,1896-01-01,12/31/1941,False


    - a README file `./data/1_raw/cities/README.txt` can give you additional information on these files.
    - all other files are named with a city code (example: `./data/1_raw/cities/USW00094728.csv` for "New York") and contains historical temperature and precipitations between **1894** and **2021**, if available. They contains the following columns:
        * `Date`: day (format YYYY-mm-dd)
        * `tmax`: maximum temperature observed during the day (in fahrenheit)
        * `tmin`: minimum temperature observed during the day (in fahrenheit)
        * `prcp`: daily precipitation amount (inch per square meter, in/m²)

In [105]:
# one file with temperature and precipitation
city_example = pd.read_csv("./data/1_raw/cities/USW00094728.csv")
print('One example of a city file (USW00094728): print the first 5 rows of the dataframe')
display(city_example.head())

One example of a city file (USW00094728): print the first 5 rows of the dataframe


,Unnamed: 0,Date,tmax,tmin,prcp
0,1,1869-01-01,29.0,19.0,0.75
1,2,1869-01-02,27.0,21.0,0.03
2,3,1869-01-03,35.0,27.0,0.00
3,4,1869-01-04,37.0,34.0,0.18
4,5,1869-01-05,43.0,37.0,0.05


- Demographics
    - The `./data/1_raw/demographics/us-cities-demographics.csv` file contains demographic data (age, total population, etc.) for US cities. Data are taken in 2015. Columns are:
        * `City`: city name
        * `State`: state name (full name)
        * `Median Age`: median age of the population
        * `Male Population`: male population (integer)
        * `Female Population`: female population (integer)
        * `Total Population`: total population (integer)
        * `Number of Veterans`: number of veterans (integer)
        * `Foreign-born`: number of foreign-born (integer)
        * `Average Household Size`: average size of an household
        * `State Code`: state code (abbreviation)
        * `Race`: most represented race
        * `Count`: unknown column, should not be used

In [106]:
# one file with temperature and precipitation
demographics = pd.read_csv("./data/1_raw/demographics/us-cities-demographics.csv", delimiter=";")
print('Demographics file: print the first 5 rows of the dataframe')
display(demographics.head())

Demographics file: print the first 5 rows of the dataframe


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
1,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
2,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
3,Hampton,Virginia,35.5,66214.0,70240.0,136454,19638.0,6204.0,2.48,VA,Black or African-American,70303
4,Lakewood,Colorado,37.7,76013.0,76576.0,152589,9988.0,14169.0,2.29,CO,Hispanic or Latino,33630


##### One can find bellow some guidelines for this process:
- For cities information (temperature, precipitation, latitude, longitude and city name)
    - read a unique file with temperatures and precipitations, and filter the dates between 2011 and 2015 (included)
    - when reading this unique file, add a column with the city attributes (latitude, longitude and city name)
    - apply this process to read all files, and concatenate all DataFrames into a single one
- For demographics:
    - read the input file and keep one record per city
    - map states' abbreviations from their fullname (with `STATE2ABBREV`)
- For external data:
    - merge the 2 previously created DataFrame from cities and demographics, to get a unique output DataFrame
    - For each dataset, compute a new column with the city name, transformed without spaces and with lowercase. You can use the following function when computing this column: `"".join(city_name.lower().split())`
    - Use this column when joining the 2 DataFrames

<p align="center">
  <a>
    <img src="./figures/UpToYou.png" alt="Logo" width="200" height="280">
  </a>
</p>

#### Libraries

In [107]:
import pandas as pd
from utils import check_duplicates

#### Input files/variables

In [108]:
# inputs
date_min = "2011-01-01" # min date for the training set (fires)
date_max = "2015-12-31" # maximum date for predictions (fires)
city_info_filepath = "./data/1_raw/cities/city_info.csv"
demographics_filepath = "./data/1_raw/demographics/us-cities-demographics.csv"
dest_file = "./data/2_clean/external_data.csv"
checks = {True:"OK", False: "NOK"}

# Dict to convert state names to abbreviations (2 letter code).
STATE2ABBREV = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

### Merge all external data files into a single csv file

In [109]:
# partie Dan 
import os
#initialisation de la dataframe information qui résulte de la concaténation des informations de chaque ville
index0=os.listdir("./data/1_raw/cities/")[0]
city = pd.read_csv(f"./data/1_raw/cities/{index0}")
informations=pd.DataFrame(columns=city.columns)
#________________________________________________________
for i in os.listdir("./data/1_raw/cities/"):
    if (i!="README.txt") & (i!="city_info.csv"): #on ne s'intérresse pas à ces deux fichiers du folder
        
        #on ouvre chaque fichier contenant les infos de chaque ville
        city = pd.read_csv(f"./data/1_raw/cities/{i}")
        
        #on filtre les dates qui nous intéresse (2011/2015)
        city=city[city["Date"]<=date_max]
        city=city[city["Date"]>=date_min]
    

        #on prend les infos de chaque ville de city_infos pour les ajouter à city
        infos=city_infos[city_infos['ID']==i[:-4]]
        
#on créé des colonnes pour les attributs de city_infos qui nous intéresse dans la dataframe city
        lat=[float(infos.loc[:,"Lat"].iloc[0]) for i in range(len(city))]
        city=city.assign(Lat=lat)
        
        lon=[float(infos.loc[:,"Lon"].iloc[0]) for i in range(len(city))]
        city=city.assign(Lon=lon)
        
        Name=[str(infos.loc[:,"Name"].iloc[0]) for i in range(len(city))]
        city=city.assign(Name=Name)
        
        #on concatène toutes toutes les infos de toutes les villes ensemble
        informations=pd.concat([informations,city])

informations = informations.loc[:, ~informations.columns.str.contains('^Unnamed')]
informations

,Date,tmax,tmin,prcp,Lat,Lon,Name
43098,2011-01-01,36.0,28.0,0.00,44.9050,-123.0011,Salem
43099,2011-01-02,38.0,24.0,0.00,44.9050,-123.0011,Salem
43100,2011-01-03,39.0,22.0,0.00,44.9050,-123.0011,Salem
43101,2011-01-04,35.0,21.0,0.00,44.9050,-123.0011,Salem
43102,2011-01-05,47.0,32.0,0.08,44.9050,-123.0011,Salem
...,...,...,...,...,...,...,...
47111,2015-12-27,29.0,13.0,0.00,43.9042,-92.4917,RochesterMN
47112,2015-12-28,22.0,13.0,0.48,43.9042,-92.4917,RochesterMN
47113,2015-12-29,26.0,19.0,0.16,43.9042,-92.4917,RochesterMN
47114,2015-12-30,22.0,14.0,0.00,43.9042,-92.4917,RochesterMN


In [110]:
index_name = list(informations.columns.values).index("Name")
Name_lo=["".join(str(informations.iloc[i,index_name]).lower().split()) for i in range(len(informations))]

informations = informations.assign(Name_lo=Name_lo)

In [111]:
# partie asmaa
demographics = pd.read_csv("./data/1_raw/demographics/us-cities-demographics.csv", delimiter=";")
# affichages
# demographics.shape
# demographics.info()
# demographics[demographics['City']=='Abilene']
# demographics.describe()
# demographics.head()
# demographics.duplicated().value_counts() # without duplicates
# __________________________

demographics = demographics.drop(['Race', 'Count', 'State Code'], axis=1)

demographics.drop_duplicates(keep='first', inplace=True)
demographics.duplicated(subset=['City']).value_counts()   # one city could have multiple records 
demographics['State'] = demographics['State'].map(STATE2ABBREV)

# création d'une series contenant en indice les villes etant presentes plusieurs fois
df2 = demographics.groupby(by='City')['State'].agg('count')
i = 0
while df2.index.tolist()[i] != "Yuma":  # pas ouf mais j'ai pas trouvé mieux
    if float(df2.iloc[i]) <= 1:
        df2 = df2.drop([df2.index.tolist()[i]])

        i = i-1
    i = i+1
df2 = df2.drop([df2.index.tolist()[i]])

list_cities2 = df2.index.tolist()

In [112]:
df1 = demographics

index_city = list(df1.columns.values).index("City")
index_state = list(df1.columns.values).index("State")
Name_lo = []
for i in range(len(df1)):
    if df1.iloc[i, index_city] in list_cities2:
        string = str(df1.iloc[i, index_city])+str(df1.iloc[i, index_state])

    else:
        string = str(df1.iloc[i,index_city])

    Name_lo.append("".join(string.lower().split()))

df1=df1.assign(Name_lo=Name_lo)

In [114]:
external_data = informations.merge(df1, on="Name_lo")
external_data = external_data.drop_duplicates()
external_data

,Date,tmax,tmin,prcp,Lat,Lon,Name,Name_lo,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size
0,2011-01-01,36.0,28.0,0.00,44.9050,-123.0011,Salem,salem,Salem,OR,35.4,80839.0,83704.0,164543,9685.0,19668.0,2.69
1,2011-01-02,38.0,24.0,0.00,44.9050,-123.0011,Salem,salem,Salem,OR,35.4,80839.0,83704.0,164543,9685.0,19668.0,2.69
2,2011-01-03,39.0,22.0,0.00,44.9050,-123.0011,Salem,salem,Salem,OR,35.4,80839.0,83704.0,164543,9685.0,19668.0,2.69
3,2011-01-04,35.0,21.0,0.00,44.9050,-123.0011,Salem,salem,Salem,OR,35.4,80839.0,83704.0,164543,9685.0,19668.0,2.69
4,2011-01-05,47.0,32.0,0.08,44.9050,-123.0011,Salem,salem,Salem,OR,35.4,80839.0,83704.0,164543,9685.0,19668.0,2.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211811,2015-12-27,29.0,13.0,0.00,43.9042,-92.4917,RochesterMN,rochestermn,Rochester,MN,35.0,54934.0,57282.0,112216,6888.0,17763.0,2.55
211812,2015-12-28,22.0,13.0,0.48,43.9042,-92.4917,RochesterMN,rochestermn,Rochester,MN,35.0,54934.0,57282.0,112216,6888.0,17763.0,2.55
211813,2015-12-29,26.0,19.0,0.16,43.9042,-92.4917,RochesterMN,rochestermn,Rochester,MN,35.0,54934.0,57282.0,112216,6888.0,17763.0,2.55
211814,2015-12-30,22.0,14.0,0.00,43.9042,-92.4917,RochesterMN,rochestermn,Rochester,MN,35.0,54934.0,57282.0,112216,6888.0,17763.0,2.55


# Take Away
- Finding external data in a AI project can improve the scoring of the model
- It is often necessary to transform this external data before being able to merge it

### Pitfalls to avoid
- not checking for any duplicate values after a join/merge operation